In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("./data/energydata_complete.csv")

target_name = "Appliances"
continuous_features = [
    "lights",
    "T1", 
    "RH_1",
    "T2",
    "RH_2",
    "T3",
    "RH_3",
    "T4",
    "RH_4",
    "T5",
    "RH_5",
    "T6",
    "RH_6",
    "T7",
    "RH_7",
    "T8",
    "RH_8",
    "T9",
    "RH_9",
    "T_out",
    "RH_out",
    "Press_mm_hg",
    "Windspeed"
]
during_training = 15
after_prediction = 5
data_groups = df.shape[0] // (during_training + after_prediction)

In [3]:
X = list()
y = list()
for one_batch in range(data_groups):
    X.append(df[continuous_features].iloc[one_batch:during_training+one_batch].values.tolist())
    y.append(df[target_name].iloc[during_training+after_prediction+one_batch])

X = np.array(X)
y = np.array(y)

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, shuffle = True)

In [4]:
train_X.shape

(788, 15, 23)

In [5]:
DL_layers = [
    {
        "RNN": {
            "input_size": train_X.shape[-1],
            "hidden_size": train_X.shape[-1] // 2,
            "num_layers": 3,
            "bidirectional": True
        }
    },
    {
        "self-attention": {
            "embed_size": train_X.shape[-1] // 2 * 2,
            "target_length": train_X.shape[1],
            "target_compression_length": 3,
            "num_heads": 2
        }
    }, 
    {
        "LSTM": {
            "input_size": train_X.shape[-1] // 2 * 2,
            "hidden_size": 1,
            "num_layers": 3,
            "bidirectional": True
        }        
    },
    {
        "flatten": {}        
    },
    {
        "linear": {
            "in_features": train_X.shape[1] * 2,
            "out_features": 1
        }
    }
]

In [6]:
train_X.shape[-1] // 2 * 2

22

In [7]:
from DL_for_time_series import DL_time_series_training_flow
training_obj = DL_time_series_training_flow(
    DL_layers = DL_layers,
    loss_func = "mse",
    optimizer = "adam",
    epochs = 30,
    target_type = "regression"
)
result = training_obj.fit(
    train_X = train_X,
    train_y = train_y,
    test_X = test_X,
    test_y = test_y,
    batch_size = 16
)

sample_model(
  (model): Sequential(
    (0): RNN_block(
      (model): RNN(23, 11, num_layers=3, batch_first=True, bidirectional=True)
    )
    (1): self_attention_block(
      (model): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=110, out_features=110, bias=True)
      )
    )
    (2): LSTM_block(
      (model): LSTM(22, 1, num_layers=3, batch_first=True, bidirectional=True)
    )
    (3): Flatten(start_dim=1, end_dim=-1)
    (4): Linear(in_features=30, out_features=1, bias=True)
  )
)
Epoch:  0 Train Loss:  31670.179560546876 Test Loss:  22849.511005108172
Epoch:  1 Train Loss:  31536.877236328124 Test Loss:  22706.83916766827
Epoch:  2 Train Loss:  31315.0262109375 Test Loss:  22440.989595853367
Epoch:  3 Train Loss:  30864.714921875 Test Loss:  21910.84814453125
Epoch:  4 Train Loss:  30237.280625 Test Loss:  21408.97644981971
Epoch:  5 Train Loss:  29709.12625 Test Loss:  20969.460298978367
Epoch:  6 Train Loss:  29242.502177734375 Test Los